In [ ]:
from pdf2image import convert_from_path
import pytesseract

images = convert_from_path("HSC26-Bangla1st-Paper.pdf",dpi = 200)
bangla_text = []

for img in images:
    text = pytesseract.image_to_string(img, lang='ben')
    bangla_text.append(text)

print("\n".join(bangla_text))


In [ ]:
text = bangla_text[5:19]
for i in text:
    print(i)

In [3]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

In [4]:
llm_model = ChatOllama(model = "llama3.2",temperature = 0)

In [5]:
doc_text = [Document(i) for i in text]

In [11]:
embeddings = OllamaEmbeddings(model = "nomic-embed-text")

In [12]:
rec_obj = RecursiveCharacterTextSplitter(chunk_size= 300,chunk_overlap = 100)

In [13]:
rec_text = rec_obj.split_documents(doc_text)

In [14]:
len(rec_text)

164

In [15]:
cm = Chroma.from_documents(rec_text,embedding = embeddings)

In [16]:
db = cm.as_retriever()

In [18]:
def add_doc(docs):
    return "\n".join([i.page_content for i in docs])

In [19]:

chat_template = """
You are a multilingual assistant that answers questions based on provided story context. 

Context:{context}
Question: {question}
"""
chat_prompt = ChatPromptTemplate.from_template(chat_template)


chain = {"context":db|add_doc,"question":RunnablePassthrough()}|chat_prompt|llm_model


In [20]:
chain.invoke("who is onupom?").content 

'অনুপম হলেন গল্পের কথক।'

In [22]:
chain.invoke("অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?").content

'শস্তুনাথ'